In [ ]:
from pathlib import Path
import nglview
import ipywidgets as widgets
from ipywidgets import interactive, VBox
from IPython.display import display

import numpy as np

probes = ['PHEN', 'ACET', 'IPA', 'DMAD', 'IPO', 'PPN', 'PHEN_ACET_IPA_DMAD_IPO_PPN']

class InteractiveHotspot():
    def __init__(self):
        self.view = nglview.NGLWidget()
        self.view.add_component('aa_aligned.pdb')
        for p in probes:
            self.view.add_component('dens_' + p + '.dx')
    
    def get_component_index(self, component_name):
        """
        Given the name of the loaded file (or a partial string),
        returns the component index if loaded
        """
        all_components = self.view._ngl_component_names
        component_idx = all_components.index([i for i in all_components 
                                              if component_name in i][0])
        return component_idx
    
    def threshold_isosurface(self, probe_name, threshold):
        """
        Controls the representation of the map for specific probe
        :param probe_name: string, one of 'donor', 'acceptor', 'apolar'
        :param threshold: threshold value for displaying the hotspot map
        """
        colour_dict = {'PHEN': 'red', 'ACET': 'blue', 'IPA':'green',
                       'DMAD':'yellow', 'IPO':'violet', 'PPN':'cyan',
                      'PHEN_ACET_IPA_DMAD_IPO_PPN': 'gray'}
        comp_idx = self.get_component_index(probe_name)
        repr_params = [{'type': 'surface',
                        'params': {'opacity': 0.4,
                                   'isolevelType': 'value',
                                   'isolevel': threshold,
                                   'color': colour_dict[probe_name]}}]
        self.view.set_representations(repr_params, component=comp_idx)
        
    def hide_isosurface(self, probe_name):
        """
        Hide surface for a given probe
        """
        comp_idx = self.get_component_index(probe_name)
        repr_params = [{'type': 'surface',
                        'params': {'opacity': 0.01}}]
        self.view.set_representations(repr_params, component=comp_idx)
        
hotspot = InteractiveHotspot()

# Create 6 checkboxes and 6 sliders
checkboxes, textboxes = [], []
for p in probes:
    checkboxes.append(widgets.Checkbox(description = p))
    textboxes.append(widgets.BoundedFloatText(value = -20, min=-20, max=-1, step=0.1, description = p))
    textboxes[-1].layout.visibility = 'hidden'

# Function to update the visibility of sliders based on checkbox states
def update_textbox_visibility(change):
    checkbox_id = probes.index(change.owner.description.split()[-1])
    checkbox_name = change.owner.description.split()[-1]
    if textboxes[checkbox_id].layout.visibility == 'visible':
        textboxes[checkbox_id].layout.visibility = 'hidden'
        hotspot.hide_isosurface(checkbox_name)
    else:
        textboxes[checkbox_id].layout.visibility = 'visible'
        hotspot.threshold_isosurface(checkbox_name, textboxes[checkbox_id].value)
    

def on_textbox_change(change):
    textbox_name = change.owner.description.split()[-1]
    hotspot.threshold_isosurface(textbox_name, change.new)
    
# Attach the function to checkbox changes
for checkbox in checkboxes:
    checkbox.observe(update_textbox_visibility, 'value')

for textbox in textboxes:
    textbox.observe(on_textbox_change, names='value')    

# Create a vertical box to organize the widgets
checkboxes_box = widgets.VBox(checkboxes)
textboxes_box = widgets.VBox(textboxes)

VBox([hotspot.view, widgets.HBox([checkboxes_box, textboxes_box])])